[DIY Covid-19 Dashboard Kit](https://github.com/fsmeraldi/diy-covid19dash) (C) Fabrizio Smeraldi, 2020 ([f.smeraldi@qmul.ac.uk](mailto:f.smeraldi@qmul.ac.uk) - [web](http://www.eecs.qmul.ac.uk/~fabri/)). This notebook is released under the [GNU GPLv3.0 or later](https://www.gnu.org/licenses/).

# DIY Covid-19 Dashboard

This is my DIY Covid Dashboard :) for this I compared the data from the PHE API, comparing 'hospital admissions' and 'mechanical ventilator bed occupancy' for each day in England. 

In [3]:
from IPython.display import clear_output
import ipywidgets as wdg
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
from uk_covid19 import Cov19API

In [4]:
%matplotlib inline
# make figures larger
plt.rcParams['figure.dpi'] = 100

In [5]:
# Load JSON files and store the raw data in some variable. Edit as appropriate
with open("VBedsAndOccupancy.json", "rt") as INFILE:
    data=json.load(INFILE)

## Covid Data

This table displays the number of hospital admissions and the number of mechanical ventilator bed occupied on each date. 

In [14]:
with open("VBedsAndOccupancy.json", "rt") as INFILE:
    data=json.load(INFILE)
    
datalist=data['data']

dates=[dictionary['date'] for dictionary in datalist ]
dates.sort()

def parse_date(datestring):
    """ Convert a date string into a pandas datetime object """
    return pd.to_datetime(datestring, format="%Y-%m-%d")

startdate=parse_date(dates[0])
enddate=parse_date(dates[-1])

index=pd.date_range(startdate, enddate, freq='D')
VBedsAndOccupancydf=pd.DataFrame(index=index, columns=['admissions','MVBedsOccupied'])

for entry in datalist: # each entry is a dictionary with date, cases, hospital and deaths
    date=parse_date(entry['date'])
    for column in ['admissions', 'MVBedsOccupied']:
        if pd.isna(VBedsAndOccupancydf.loc[date, column]): 
            value= float(entry[column]) if entry[column]!=None else 0.0
            VBedsAndOccupancydf.loc[date, column]=value
            
# fill in any remaining "holes" due to missing dates
VBedsAndOccupancydf.fillna(0.0, inplace=True)
            
VBedsAndOccupancydf

,admissions,MVBedsOccupied
2020-03-19,586.0,0.0
2020-03-20,1277.0,0.0
2020-03-21,2054.0,0.0
2020-03-22,2913.0,0.0
2020-03-23,4041.0,0.0
...,...,...
2023-10-09,1079594.0,0.0
2023-10-10,1080271.0,0.0
2023-10-11,1080814.0,0.0
2023-10-12,1081331.0,0.0


## Current data

Clicking on the button below just generates updated data for the graph below :)

In [15]:
from IPython.display import clear_output
import ipywidgets as wdg
import pandas as pd
import matplotlib.pyplot as plt

In [16]:
%matplotlib inline
# make figures larger
plt.rcParams['figure.dpi'] = 100

In [17]:
def access_api(button):
    # Ignore the button parameter
    #
    # put code for polling the API here
    #
    # after all is done, you can switch the icon on the button to a "check" sign
    # and optionally disable the button - it won't be needed again.
    apibutton.icon="check"
    apibutton.disabled=True

# see the doc for the parameters    
apibutton=wdg.Button(
    description='Refresh data',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click to download current Public Health England data',
    icon='download' # (FontAwesome names without the `fa-` prefix)
)

# register the callback function with the button
apibutton.on_click(access_api)

# this is an iPython function that generalises print for Jupyter Notebooks; we use it to 
# display the widgets
display(apibutton)

Button(description='Refresh data', icon='download', style=ButtonStyle(), tooltip='Click to download current Pu…

## Graphs and Analysis

This graph shows the linear and log scale for those admitted to hospital for Covid19 and those who were admitted and placed in a monitored ventilated bed. please interact with the graphs below :)

In [18]:
VBedsAndOccupancydf=pd.read_pickle("VBedsAndOccupancydf.pkl")

In [12]:
series=wdg.SelectMultiple(
   options=['admissions', 'MVBedsOccupied'],
    value=['admissions', 'MVBedsOccupied'],
    rows=2,
    description='Stats:',
    disabled=False
)

scale=wdg.RadioButtons(
    options=['linear', 'log'],
#   value='pineapple', # Defaults to 'pineapple'
#   layout={'width': 'max-content'}, # If the items' names are long
    description='Scale:',
    disabled=False
)

controls=wdg.HBox([series, scale])

def VBedsAndOccupancy_graph(gcols, gscale):
    if gscale=='linear':
        logscale=False
    else:
        logscale=True
    ncols=len(gcols)
    if ncols>0:
        VBedsAndOccupancydf[list(gcols)].plot(logy=logscale)
        plt.show() # important - graphs won't update if this is missing 
    else:
        print("Click to select data for graph")
        print("(CTRL-Click to select more than one category)")

# keep calling timeseries_graph(gcols=value_of_series, gscale=value_of_scale); 
# capture output in widget graph   
graph=wdg.interactive_output(VBedsAndOccupancy_graph, {'gcols': series, 'gscale': scale})

display(controls, graph)

Output()

Hopefully this data can be insightful. An observation to be made is that in the graphs created, there is a small percentage of people who were admitted and placed on mechanical ventilator beds. Looking at the graphs there is an increase in the number of patients in mechanical ventilator beds in January of 2021.

Thanks for reading :)

**Author and Copyright Notice** Remember if you deploy this dashboard as a Binder it will be publicly accessible. Take credit for your work! Also acknowledge your sources: Based on UK Government [data](https://coronavirus.data.gov.uk/) published by [Public Health England](https://www.gov.uk/government/organisations/public-health-england) and on the [DIY Covid Dashboard Kit](https://github.com/fsmeraldi/diy-covid19dash), Copyright (C) Fabrizio Smeraldi 2020,2023. Released under the [GNU GPLv3.0 or later](https://www.gnu.org/licenses/).